***GENERATED CODE FOR eligdenresolutionrecommendationautoml PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
from pyspark.ml.feature import StringIndexer
import json
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'}], 'feature': 'ReasonDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
                                                'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ReasonDate'}, {'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '77917.47', 'stddev': '29095.18', 'min': '11042', 'max': 'Q9969', 'missing': '0', 'distinct': '107'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '5.17', 'stddev': '11.22', 'min': '0000', 'max': 'RT', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '0.05', 'stddev': '1.7', 'min': '0000', 'max': '59', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier2'}, {'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'A04.72', 'max': 'Z95.818', 'missing': '0', 'distinct': '365'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'A41.9', 'max': 'Z99.81', 'missing': '0', 'distinct': '399'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'U', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'CAT Scans', 'max': 'Supplies', 'missing': '0', 'distinct': '27'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Cardiac Surgery', 'max': 'Vascular Surgery', 'missing': '0', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = transformationDF.drop('payerid')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'Work Comp', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerCategory'}, {'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Work Comp', 'missing': '0', 'distinct': '13'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerGroup'}, {'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Wellcare Staywell', 'missing': '0', 'distinct': '61'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerName'}, {'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Ambetter Commercial HMO', 'max': 'United Healthcare Medicare Advantage', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup1'}, {'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Wellcare/Staywell', 'missing': '0', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup2'}, {'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'United Healthcare', 'missing': '0', 'distinct': '15'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerCategory'}, {'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'UHC', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerGroup'}, {'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Wellcare Staywell', 'missing': '0', 'distinct': '47'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerName'}, {'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'AvMed', 'max': 'United Healthcare Medicare Advantage', 'missing': '0', 'distinct': '13'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup1'}, {'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Wellcare/Staywell', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup2'}, {'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '77917.47', 'stddev': '29095.18', 'min': '11042', 'max': 'Q9969', 'missing': '0', 'distinct': '107'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemName'}, {'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': '3D RENDER W/O POSTPROCESS', 'max': 'X-RAY KNEE COMPLETE; 4 OR MORE VIEWS', 'missing': '0', 'distinct': '97'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemCategory'}, {'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceLocation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Home', 'max': 'Urgent Care Facility', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceLocation'}, {'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceLocation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reasonType', 'transformation_label': 'String Indexer'}], 'feature': 'reasonType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Denial', 'max': 'Denial', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reasonType'}, {'feature_label': 'reasonType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reasonType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'}], 'feature': 'Reasoncode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '29.72', 'stddev': '21.98', 'min': '109', 'max': '33', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reasoncode'}, {'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reasoncode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '', 'stddev': '', 'min': 'Benefits Max/Exh', 'max': 'Eligibility', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonCategory'}, {'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDescription', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '1202', 'mean': '', 'stddev': '', 'min': 'Charges are covered under a capitation agreement/managed care plan.',
                                                                                                                                                                                                                                                                                    'max': 'This is a work-related injury/illness and thus the liability of the Worker`s Compensation Carrier.', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDescription'}, {'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDescription')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'label_resolution', 'transformation_label': 'String Indexer'}], 'feature': 'label_resolution', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1202', 'mean': '3.08', 'stddev': '2.28', 'min': '0', 'max': '7', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'label_resolution'}, {'feature_label': 'label_resolution', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('label_resolution')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


def randomforestclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    rf_model = RandomForestClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    rf_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % rf_model.score(X_test, y_test))

    data = {'model': rf_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_BOND_CLINIC', 'optionalDB': 'NextGen_BOND_CLINIC', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT\r\n\t\r\n\t\t\t\tcd.ChargeDetailID, \r\n\t\t\t\ttr.ReasonDate,\r\n\t\t\t\tcptc.CPTCode,\r\n\t\t\t\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\t\t\t\tCASE WHEN cd.CPTModifier2 IS NOT NULL THEN cd.CPTModifier2 ELSE '0000' END as CptModifier2,\r\n\t\t\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\t\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\t\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\t            cd.Amount,\r\n\t\t\t\td.DepartmentCategory,\r\n\t\t\t\ts.SpecialityCategory,\r\n\t\t\t\tpy.payerid,\r\n\t\r\n\t\t\t\tCASE WHEN py.PayerCategory IS NOT NULL THEN py.PayerCategory ELSE 'NOT AVAILABLE' END as PrimaryPayerCategory ,\r\n\t\t\t\tCASE WHEN py.PayerGroup IS NOT NULL THEN py.PayerGroup ELSE 'NOT AVAILABLE' END as PrimaryPayerGroup ,\r\n\t\t\t\tCASE WHEN py.PayerName IS NOT NULL THEN py.PayerName ELSE 'NOT AVAILABLE' END as PrimaryPayerName ,\r\n\t\t\t\tCASE WHEN py.PayerSubGroup1 IS NOT NULL THEN py.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup1 ,\r\n\t\t\t\tCASE WHEN py.PayerSubGroup2 IS NOT NULL THEN py.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup2 ,\r\n\t\r\n\t\t\t\tCASE WHEN py1.PayerCategory IS NOT NULL THEN py1.PayerCategory ELSE 'NOT AVAILABLE' END as SecondPayerCategory ,\r\n\t\t\t\tCASE WHEN py1.PayerGroup IS NOT NULL THEN py1.PayerGroup ELSE 'NOT AVAILABLE' END as SecondPayerGroup ,\r\n\t\t\t\tCASE WHEN py1.PayerName IS NOT NULL THEN py1.PayerName ELSE 'NOT AVAILABLE' END as SecondPayerName ,\r\n\t\t\t\tCASE WHEN py1.PayerSubGroup1 IS NOT NULL THEN py1.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup1 ,\r\n\t\t\t\tCASE WHEN py1.PayerSubGroup2 IS NOT NULL THEN py1.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup2 ,\r\n\t\r\n\t\t\t\tCASE WHEN si.ServiceItemName IS NOT NULL THEN si.ServiceItemName ELSE 'NOT AVAILABLE' END as ServiceItemName ,\r\n\t\t\t\tCASE WHEN si.ServiceItemCategory IS NOT NULL THEN si.ServiceItemCategory ELSE 'NOT AVAILABLE' END as ServiceItemCategory,\r\n\t\t\t\tCASE WHEN sl.ServiceLocation IS NOT NULL THEN sl.ServiceLocation ELSE 'NOT AVAILABLE' END as ServiceLocation,\r\n\r\n\t\t\ttrc.reasonType,\r\n\t\t\ttrc.TransactionReasonCodeID,\r\n\t\t\ttrc.Reasoncode,\r\n\t\t\ttrc.ReasonCategory,\r\n\t\t\ttrc.ReasonDescription\r\n\t\t\t\r\n\r\n\t,CASE \r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('3','9','12','47','50','56','73','84') THEN '1'\r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('4','44','54','62','66','67','76') THEN '2'\r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('5','45') THEN '3'\r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('6','11','46','48','52','58','70','71','75','78','28512') THEN '4'\r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('7','51','59','60','69','72') THEN '5'\r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('8','13','41','42','43','61','63','64','65','68','89','789') THEN '6'\r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('10','14','15','57','74','34271') THEN '7'\r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('16','49','77','81','82','86','87','30899') THEN '8'\r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('53','55','83','88') THEN '9'\r\n         WHEN  t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID IN ('79','80','85','90') THEN '10'\r\n\r\n  \t\tELSE '0' END AS label_resolution\r\n\r\n\r\n\tfrom ChargeDetail cd\r\n\tinner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\tinner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID  \r\n\tleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = t.PayerID \r\n\tinner join Bill b WITH (NOLOCK) on e.EncounterID = b.EncounterID and b.BillID = t.BillID \r\n\tinner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID AND T.TransactionDate>=TR.ReasonDate\r\n\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\tinner join Payer py WITH (NOLOCK) on py.PayerID = t.PayerID\r\n\tleft join Payer py1 WITH (NOLOCK) on py1.PayerID = cd.SecondaryPayerID\r\n\tleft join Payer py2 WITH (NOLOCK) on py2.PayerID = cd.TertiaryPayerID\r\n\tinner join ServiceItem si WITH (NOLOCK) on cd.ServiceItemID = si.ServiceItemID\r\n\tleft outer join ServiceLocation sl WITH (NOLOCK) on cd.ServiceLocationID = sl.ServiceLocationID\r\n\t\t\t\r\n\t\t\tinner join Transactiontype tt on tt.transactiontypeid = t.transactiontypeid\r\n\t\t\tleft outer join  Task tsk WITH (NOLOCK) on e.encounterid = tsk.encounterid and e.facilityid = tsk.facilityid and tsk.createddate >= tr.reasondate \r\n\t\t\tleft outer join  TaskCompletionReason tcr WITH (NOLOCK) on tsk.TaskCompletionReasonID = tcr.TaskCompletionReasonID  \r\n\t\t\tleft outer join  Taskstatus ts on tsk.TaskStatusID=ts.TaskStatusID\r\n\t\t\tleft outer join  TaskType tskt on tsk.TaskTypeID=tskt.TaskTypeID\r\n\r\nWHERE\t\tcd.Amount >.01\r\n\t\t\tand trc.ReasonCode IN ('19','20','21','22','24','26','27','28','30','31','32','33','109','166','200')\r\n\t\t\tand trc.reasonType = 'Denial'\r\n\t\t\tand tr.ReasonDate between '2018-01-01' and '2020-12-31'\r\n\t\t\tand ts.TaskStatusCategory ='Completed'\r\n\t\t\tand tcr.TaskCompletionReason !='NULL' \r\n\t\t\tand tcr.TaskCompletionReasonID NOT IN ('99','100','101') \r\n\t\t\tand tsk.CreatedDate >= tr.ReasonDate\r\n\t\t\tand TransactionDate >= tsk.CompletedDate \r\n\t\t\tand tskt.TaskTypeCategory='AR'\r\n\t\t\tand py.PayerCategory!='Self Pay'", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "00B05C22-818A-4B4C-BA43-0957A6F5E3C9", "max": "FF63525C-4AEF-4EBA-9A87-CA526130762A", "missing": "0", "distinct": "1100"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "Extract Date"}], "feature": "ReasonDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ReasonDate"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "77917.47", "stddev": "29095.18", "min": "11042", "max": "Q9969", "missing": "0", "distinct": "107"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "5.17", "stddev": "11.22", "min": "0000", "max": "RT", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "CptModifier2", "transformation_label": "String Indexer"}], "feature": "CptModifier2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "0.05", "stddev": "1.7", "min": "0000", "max": "59", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier2"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "A04.72", "max": "Z95.818", "missing": "0", "distinct": "365"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "A41.9", "max": "Z99.81", "missing": "0", "distinct": "399"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1202", "mean": "71.84", "stddev": "14.46", "min": "3", "max": "103", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "F", "max": "U", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "1202", "mean": "167.29", "stddev": "466.75", "min": "0.0", "max": "8617.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "CAT Scans", "max": "Supplies", "missing": "0", "distinct": "27"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Cardiac Surgery", "max": "Vascular Surgery", "missing": "0", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"feature_label": "payerid", "transformation_label": "String Indexer"}], "feature": "payerid", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "027330C1-DCB1-4CCE-8D3D-6D4DC325CFAF", "max": "FC574DD7-0F1E-4350-A8F8-A5F7CB490D60", "missing": "0", "distinct": "65"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "payerid"}, {"transformationsData": [{"feature_label": "PrimaryPayerCategory", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "Work Comp", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerGroup", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Aetna", "max": "Work Comp", "missing": "0", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerGroup"}, {"transformationsData": [{"feature_label": "PrimaryPayerName", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Wellcare Staywell", "missing": "0", "distinct": "61"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerName"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Ambetter Commercial HMO", "max": "United Healthcare Medicare Advantage", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup1"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Aetna", "max": "Wellcare/Staywell", "missing": "0", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup2"}, {"transformationsData": [{"feature_label": "SecondPayerCategory", "transformation_label": "String Indexer"}], "feature": "SecondPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "United Healthcare", "missing": "0", "distinct": "15"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerCategory"}, {"transformationsData": [{"feature_label": "SecondPayerGroup", "transformation_label": "String Indexer"}], "feature": "SecondPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Aetna", "max": "UHC", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerGroup"}, {"transformationsData": [{"feature_label": "SecondPayerName", "transformation_label": "String Indexer"}], "feature": "SecondPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Wellcare Staywell", "missing": "0", "distinct": "47"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerName"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "AvMed", "max": "United Healthcare Medicare Advantage", "missing": "0", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup1"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Aetna", "max": "Wellcare/Staywell", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup2"}, {"transformationsData": [{"feature_label": "ServiceItemName", "transformation_label": "String Indexer"}], "feature": "ServiceItemName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "77917.47", "stddev": "29095.18", "min": "11042", "max": "Q9969", "missing": "0", "distinct": "107"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemName"}, {"transformationsData": [{"feature_label": "ServiceItemCategory", "transformation_label": "String Indexer"}], "feature": "ServiceItemCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "3D RENDER W/O POSTPROCESS", "max": "X-RAY KNEE COMPLETE; 4 OR MORE VIEWS", "missing": "0", "distinct": "97"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemCategory"}, {"transformationsData": [{"feature_label": "ServiceLocation", "transformation_label": "String Indexer"}], "feature": "ServiceLocation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Home", "max": "Urgent Care Facility", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceLocation"}, {"transformationsData": [{"feature_label": "reasonType", "transformation_label": "String Indexer"}], "feature": "reasonType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Denial", "max": "Denial", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reasonType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TransactionReasonCodeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1202", "mean": "1158.09", "stddev": "73.75", "min": "884", "max": "1244", "missing": "0"}, "updatedLabel": "TransactionReasonCodeID"}, {"transformationsData": [{"feature_label": "Reasoncode", "transformation_label": "String Indexer"}], "feature": "Reasoncode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "29.72", "stddev": "21.98", "min": "109", "max": "33", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reasoncode"}, {"transformationsData": [{"feature_label": "ReasonCategory", "transformation_label": "String Indexer"}], "feature": "ReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Benefits Max/Exh", "max": "Eligibility", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonCategory"}, {"transformationsData": [{"feature_label": "ReasonDescription", "transformation_label": "String Indexer"}], "feature": "ReasonDescription", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "", "stddev": "", "min": "Charges are covered under a capitation agreement/managed care plan.", "max": "This is a work-related injury/illness and thus the liability of the Worker`s Compensation Carrier.", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDescription"}, {"transformationsData": [{"feature_label": "label_resolution", "transformation_label": "String Indexer"}], "feature": "label_resolution", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1202", "mean": "3.08", "stddev": "2.28", "min": "0", "max": "7", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "label_resolution"}, {"feature": "ReasonDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "1202", "mean": "16.71", "stddev": "8.54", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "ReasonDate_dayofmonth"}, {"feature": "ReasonDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "1202", "mean": "5.25", "stddev": "3.15", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "ReasonDate_month"}, {"feature": "ReasonDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "1202", "mean": "2018.57", "stddev": "0.53", "min": "2018", "max": "2020", "missing": "0"}, "updatedLabel": "ReasonDate_year"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "9.65", "stddev": "19.42", "min": "0.0", "max": "106.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "CptModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "0.65", "stddev": "2.01", "min": "0.0", "max": "20.0", "missing": "0"}, "updatedLabel": "CptModifier_stringindexer..."}, {"feature": "CptModifier2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1202", "mean": "0.0", "stddev": "0.03", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "CptModifier2_stringindexe..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "77.14", "stddev": "95.94", "min": "0.0", "max": "364.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "89.47", "stddev": "107.06", "min": "0.0", "max": "398.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1202", "mean": "0.41", "stddev": "0.49", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "2.44", "stddev": "4.35", "min": "0.0", "max": "26.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "2.95", "stddev": "3.69", "min": "0.0", "max": "22.0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PrimaryPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "0.98", "stddev": "2.2", "min": "0.0", "max": "17.0", "missing": "0"}, "updatedLabel": "PrimaryPayerCategory_stri..."}, {"feature": "PrimaryPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "0.59", "stddev": "1.52", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "PrimaryPayerGroup_stringi..."}, {"feature": "PrimaryPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "6.68", "stddev": "10.94", "min": "0.0", "max": "60.0", "missing": "0"}, "updatedLabel": "PrimaryPayerName_stringin..."}, {"feature": "PrimaryPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "0.94", "stddev": "2.22", "min": "0.0", "max": "15.0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup1_str..."}, {"feature": "PrimaryPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "2.76", "stddev": "3.78", "min": "0.0", "max": "22.0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup2_str..."}, {"feature": "SecondPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "1.22", "stddev": "2.07", "min": "0.0", "max": "14.0", "missing": "0"}, "updatedLabel": "SecondPayerCategory_strin..."}, {"feature": "SecondPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "1.04", "stddev": "1.7", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "SecondPayerGroup_stringin..."}, {"feature": "SecondPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "3.56", "stddev": "7.16", "min": "0.0", "max": "46.0", "missing": "0"}, "updatedLabel": "SecondPayerName_stringind..."}, {"feature": "SecondPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "0.91", "stddev": "1.51", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup1_stri..."}, {"feature": "SecondPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "1.46", "stddev": "2.86", "min": "0.0", "max": "20.0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup2_stri..."}, {"feature": "ServiceItemName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "9.65", "stddev": "19.42", "min": "0.0", "max": "106.0", "missing": "0"}, "updatedLabel": "ServiceItemName_stringind..."}, {"feature": "ServiceItemCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "7.34", "stddev": "16.68", "min": "0.0", "max": "96.0", "missing": "0"}, "updatedLabel": "ServiceItemCategory_strin..."}, {"feature": "ServiceLocation_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "0.08", "stddev": "0.36", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "ServiceLocation_stringind..."}, {"feature": "reasonType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1202", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "reasonType_stringindexer_..."}, {"feature": "Reasoncode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "0.71", "stddev": "1.28", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "Reasoncode_stringindexer_..."}, {"feature": "ReasonCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1202", "mean": "0.55", "stddev": "0.89", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "ReasonCategory_stringinde..."}, {"feature": "ReasonDescription_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "0.71", "stddev": "1.28", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "ReasonDescription_stringi..."}, {"feature": "label_resolution_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1202", "mean": "1.77", "stddev": "1.8", "min": "0.0", "max": "8.0", "missing": "0"}, "updatedLabel": "label_resolution_stringin..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=randomforestclassifier(autofe, ["CPTCode_stringindexer", "CptModifier_stringindexer", "CptModifier2_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "Age", "Gender_stringindexer", "Amount", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "PrimaryPayerCategory_stringindexer", "PrimaryPayerGroup_stringindexer", "PrimaryPayerName_stringindexer", "PrimaryPayerSubGroup1_stringindexer", "PrimaryPayerSubGroup2_stringindexer", "SecondPayerCategory_stringindexer", "SecondPayerGroup_stringindexer", "SecondPayerName_stringindexer", "SecondPayerSubGroup1_stringindexer", "SecondPayerSubGroup2_stringindexer", "ServiceItemName_stringindexer", "ServiceItemCategory_stringindexer", "ServiceLocation_stringindexer", "reasonType_stringindexer", "TransactionReasonCodeID", "Reasoncode_stringindexer", "ReasonCategory_stringindexer", "ReasonDescription_stringindexer", "ReasonDate_dayofmonth", "ReasonDate_month", "ReasonDate_year"], "label_resolution_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
